<a href="https://colab.research.google.com/github/smduarte/RC2021-labs/blob/main/aula6/lab_cnss_2_sw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes de Computadores
## Computer Networks


## Aula 6

### Goal
In this lab, students will get a better understanding of how packet switching network properties impact the end-to-end performance of sending information from one node to another. 

### Initial assumptions
All links are perfect and never corrupt or lose packets. Nodes never crash or lose packets. All computations in CNSS are performed instantly, without any delay.

Network configurations use links with the same characteristics: bandwidth of 2 Mbps, or 2,000,000 bps, and a propagation time, or latency, of 20 ms, since they have 4000 Km each (4000 Km / 200,000 Km per second = 4 x 10^3 / 2 x 10^5 Km per second = 2 x 10^-2 s = 20 ms).

Three network configurations will be used: [config1.1.txt](https://github.com/smduarte/RC2021-labs/blob/main/aula6/configs/config1.1.txt), [config1.2.txt](https://github.com/smduarte/RC2021-labs/blob/main/aula6/configs/config1.2.txt) and 
[config1.3.txt](https://github.com/smduarte/RC2021-labs/blob/main/aula6/configs/config1.3.txt), depicted below.



![](https://github.com/jlegatheaux/RC2020-assignments/blob/master/assignment-1/figs/config1.1-3.png?raw=1)

In [ ]:
%%writefile config1.1.txt
# A network with a sender node and a receiver node interconnected
# by a direct link. The link has 2 Mbps bandwidth and 20 ms latency


# uncomment if you want to see control algorithms traces
# parameter trace 


Node 0 1 cnss.lib.EndSystemControl NaifSWSender 10
Node 1 1 cnss.lib.EndSystemControl SWReceiver

Link 0.0 1.0 2000000 20 0.0 0.0

Writing config1.1.txt


In [ ]:
%%writefile config1.2.txt
# A network with a sender node and a receiver node interconnected
# by a switch. Both links have 2 Mbps bandwidth and 20 ms latency


# uncomment if you want to see control algorithms traces
# parameter trace   

Node 0 1 cnss.lib.EndSystemControl NaifSWSender 10
Node 1 1 cnss.lib.EndSystemControl SWReceiver
Node 2 2 cnss.lib.FloodingSwitch cnss.lib.EmptyApp

Link 0.0 2.0 2000000 20 0.0 0.0
Link 1.0 2.1 2000000 20 0.0 0.0

Writing config1.2.txt


In [ ]:
%%writefile config1.3.txt
# A network with a sender node and a receiver node interconnected
# by two switches. All links have 2 Mbps bandwidth and 20 ms latency

# uncomment if you want to see control algorithms traces
# parameter trace   

Node 0 1 cnss.lib.EndSystemControl NaifSWSender 10
Node 1 1 cnss.lib.EndSystemControl SWReceiver
Node 2 2 cnss.lib.FloodingSwitch cnss.lib.EmptyApp
Node 3 2 cnss.lib.FloodingSwitch cnss.lib.EmptyApp

Link 0.0 2.0 2000000 20 0.0 0.0
Link 2.1 3.0 2000000 20 0.0 0.0
Link 3.1 1.0 2000000 20 0.0 0.0

Writing config1.3.txt


### Data Transmission with Flow Control

In real networks several problems may arise, like packets being lost or delivered out of order to receiver nodes. To address these issues, protocols include methods for flow control, which rely on signals sent by receivers (or the network) to senders, telling them to stop, refraining from sending packets, or to continue sending them. 

### Stop and Wait Flow Control

The simplest method of flow control is known as "Stop & Wait" flow control or S&W for short. S&W is also the name of the protocol that relies on this method. 

S&W is a very simple protocol. Each time the sender sends a packet, it will enter a waiting phase, up to reception of a signal from the receiver meaning that it received the packet and it is ready for the next one. These small signal packets are known as ***acknowledgement*** packets or *ACK* packets.

### Naif S&W Sender

The code below implements a simplistic S&W Sender node in CNSS.

The protocol is implemented as follows:

1. The sender receives, as an argument in the config file, the number of data blocks to be sent to the receiver;
2. The data blocks are all of same constant size;
3. The block begins with an integer number (starting at 0); the rest of the data left blank.
4. On each 1 ms tick of the clock, the sender tries to send the next block, unless it is waiting for the previous block.
5. When the sender receives a packet it is assumed it is the *ack* of the last packet sent, allowing the sender to proceed and send the next block. 
5. When the last block is confirmed, the sender prints some statistics.

In [ ]:
%%writefile NaifSWSender.java

import java.nio.*;
import cnss.lib.*;
import cnss.simulator.*;

public class NaifSWSender extends AbstractApplicationAlgorithm {
    
    private static final int RECEIVER = 1; //ID of receiver node
    
    public static int BLOCKSIZE = 10000; // 10000*8 = 80000 bits
    public static int TOTAL_PACKETSIZE = BLOCKSIZE+Packet.HEADERSIZE; // 10000*8 = 80160 bits

    public NaifSWSender() {
        super(true, "naif-sw-sender");
    }

    int totalBlocks;
    int nextBlock;
    int totSent;
    int bytesSent;
    int startTime;

    boolean maySend;
    
    public int initialise(int now, int node_id, Node self, String[] args) {
        super.initialise(now, node_id, self, args);
        if ( args.length != 1 ) {
            log(now, "ERROR: files-sender: missing argument totalBlocks "+now+"\n\n");
            System.exit(-1);
        }

        log(now, "starting");

        totalBlocks = Integer.valueOf(args[0]);
        nextBlock = 0;
        totSent = 0;
        bytesSent = 0;
        startTime = now;

        maySend = true;
        return 1;
    }

    public void on_clock_tick( int now) {
        if (maySend && nextBlock < totalBlocks)
        sendBlock( now, nextBlock);
    }
    
    private void sendBlock( int now, int block) {
        byte[] payload = ByteBuffer.allocate(TOTAL_PACKETSIZE).putInt(0, block).array();
        
        self.send( self.createDataPacket( RECEIVER, payload ));
        
        totSent++;
        bytesSent += payload.length;
        maySend = false;
        log(now, String.format("Sent block %d (%d bytes)", block, payload.length));		
    }
    
    public void on_receive(int now, DataPacket p) {
        int ackedBlock = ByteBuffer.wrap(p.getPayload()).getInt(0);
        
        log(now, String.format("Got ack for block: %d", ackedBlock));
        
        if( nextBlock == totalBlocks - 1)
            showState( now );
        else {
            nextBlock++;
            maySend = true;
        }
    }

    public void showState(int now) {
        int transferTime = now - startTime;
        int e2eTransferRate = bytesSent * 8 / (transferTime);

        System.out.printf("---------------------------------------------\n");
        System.out.printf("%s Blocks sent: %d\n", name, totSent);
        System.out.printf("%s Bytes sent: %d\n", name, bytesSent);
        System.out.printf("%s Transfer time: %d ms\n", name, transferTime);
        System.out.printf("%s EndToEnd rate: %d Kbps\n", name, e2eTransferRate);		
        System.out.printf("---------------------------------------------\n");
    }
}

Overwriting NaifSWSender.java


### Receiver

The receiver node is very simple. 

It just awaits the arrival of data blocks and handles each received data block, by sending an ack packet containing the received block number back to the sender node.

In [ ]:
%%writefile SWReceiver.java

import java.nio.*;
import cnss.lib.*;
import cnss.simulator.*;

public class SWReceiver extends AbstractApplicationAlgorithm {
    
    public SWReceiver() {
        super(true, "sw-receiver");
    }

    public int initialise(int now, int node_id, Node self, String[] args) {
        super.initialise(now, node_id, self, args);
        log(0, "starting...");
        return 0;
    }

    public void on_receive(int now, DataPacket p) {
        int block = ByteBuffer.wrap(p.getPayload()).getInt(0);
        
        log(now, String.format("got block: %d", block));
        
        byte[] ackPayload = ByteBuffer.allocate(Integer.BYTES).putInt(0, block).array();
        
        self.send(self.createDataPacket(p.getSource(), ackPayload));
    }
} 

Overwriting SWReceiver.java



### Execution

In this simulation, using configuration [config1.1](https://github.com/smduarte/RC2021-labs/blob/main/aula6/configs/config1.1.txt), the sender and receiver are connected by a dedicated link.

In [ ]:
%%bash 

# Fetch the CNSS repository and compile it
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java

javac -cp .:cnss-classes *.java
java -cp .:cnss-classes cnss.simulator.Simulator config1.1.txt

Already up to date.
Loading configuration : config1.1.txt
Reading file
Created Node 0: 1 interf.s, ctr code: cnss.lib.EndSystemControl app code: NaifSWSender
Created Node 1: 1 interf.s, ctr code: cnss.lib.EndSystemControl app code: SWReceiver
Added link to node 0 - Link (Node1:0 I1:0)<-->(Node2:1 I2:0) bwd: 2000000 bps lat: 20 ms error: 0.0 jit: 0.0 up
Added link to node 1 - Link (Node1:0 I1:0)<-->(Node2:1 I2:0) bwd: 2000000 bps lat: 20 ms error: 0.0 jit: 0.0 up

simulation starts - first processing step with clock = 0

log: naif-sw-sender time 0 node 0 starting
log: sw-receiver time 0 node 1 starting...
log: naif-sw-sender time 1 node 0 Sent block 0 (10020 bytes)
log: sw-receiver time 61 node 1 got block: 0
log: naif-sw-sender time 81 node 0 Got ack for block: 0
log: naif-sw-sender time 81 node 0 Sent block 1 (10020 bytes)
log: sw-receiver time 141 node 1 got block: 1
log: naif-sw-sender time 161 node 0 Got ack for block: 1
log: naif-sw-sender time 161 node 0 Sent block 2 (10020 bytes

### Exercise #1

Run simulations using the other two configs: [config1.2](https://github.com/smduarte/RC2021-labs/blob/main/aula6/configs/config1.2.txt) and [config1.3](https://github.com/smduarte/RC2021-labs/blob/main/aula6/configs/config1.3.txt)


Confirm that the simulation results match the end-to-end rate obtained analytically based
on the topology and link characteristics of the simulated network.

### Exercise #2

The provided implementation is too simplistic (naive). In more realistic networking conditions, packet loss is not zero. The provided sender is missing a timeout mechanism and will fail to complete the data transfer if packets are lost.

Modify the S&W sender and receiver nodes to account for packet loss and perform block retransmissions.

To implement a timeout mechanism, you can use the built-in mechanism in CNSS, or you
can override the **maySend** flag if too many ticks have elapsed without an ack.

Suggestion. Start with a fixed timeout value. Then, use a dynamic timout, by adapting the technique from the previous lab class to estimate the RTT.

To test, add some packet loss (< 15%) to the link definitions.